In [1]:
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.sep.join(cwd_split[:cwd_split.index('mitsuba3-differentiable-heightfield-rendering')])
sys.path.insert(0, f"{root}/mitsuba3/build/python")

In [2]:
import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_rgb')
import numpy as np

## Scene

In [3]:
mi.cornell_box()

{'type': 'scene',
 'integrator': {'type': 'path', 'max_depth': 8},
 'sensor': {'type': 'perspective',
  'fov_axis': 'smaller',
  'near_clip': 0.001,
  'far_clip': 100.0,
  'focus_distance': 1000,
  'fov': 39.3077,
  'to_world': [[-1, 0, 0, 0],
   [0, 1, 0, 0],
   [0, 0, -1, 3.9],
   [0, 0, 0, 1]],
  'sampler': {'type': 'independent', 'sample_count': 64},
  'film': {'type': 'hdrfilm',
   'width': 256,
   'height': 256,
   'rfilter': {'type': 'gaussian'},
   'pixel_format': 'rgb',
   'component_format': 'float32'}},
 'white': {'type': 'diffuse',
  'reflectance': {'type': 'rgb', 'value': [0.885809, 0.698859, 0.666422]}},
 'green': {'type': 'diffuse',
  'reflectance': {'type': 'rgb', 'value': [0.105421, 0.37798, 0.076425]}},
 'red': {'type': 'diffuse',
  'reflectance': {'type': 'rgb', 'value': [0.570068, 0.0430135, 0.0443706]}},
 'light': {'type': 'rectangle',
  'to_world': [[0.23, 0, 0, 0],
   [0, -8.30516e-09, -0.19, 0.99],
   [0, 0.19, -8.30516e-09, 0.01],
   [0, 0, 0, 1]],
  'bsdf': {'

In [4]:
scene = mi.load_dict(mi.cornell_box())

In [5]:
img = mi.render(scene)

In [6]:
mi.Bitmap(img)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [10]:
sampler = mi.load_dict({"type": "independent"})
sampler.seed(7, 1)
sensor = scene.sensors()[0]
ray, _ = sensor.sample_ray(0, sampler.next_1d(), sampler.next_2d(), sampler.next_2d())
ray_o = ray.o

In [11]:
def compute_F(t): 
    # Define ray
    Tdir = dr.normalize(mi.Vector3f(1, 1, 0))
   
    ray = mi.Ray3f(ray_o, dr.normalize(Tdir * t - ray_o))

    its = scene.ray_intersect(ray)

    F = dr.dot(its.n, dr.normalize(ray.d - its.p))
    return F

In [14]:
t = mi.Float(0.1)
dr.enable_grad(t)
F = compute_F(t)
dr.forward_from(t)
print(f"F = {F} grad_F = {dr.grad(F)}")

F = [0.0032982234843075275] grad_F = [0.03282131999731064]


In [15]:
compute_F(t+0.001)

[0.0033310619182884693]